In [1]:
%matplotlib inline
import openml as oml
import numpy as np
import pandas as pd
import sys
import math
from scipy.stats import norm
from matplotlib import pyplot
import sklearn.tree
import sklearn.preprocessing
from sklearn.externals.joblib import Parallel, delayed
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from openmlstudy14.preprocessing import ConditionalImputer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Step 1: Get datasets from OpenML
- Only classification datasets
- Only active (verified) datasets

In [2]:
# Get all OpenML datasets
status_type = 'active'
openml_list = oml.datasets.list_datasets(status=status_type) # Returns a dict
datalist = pd.DataFrame.from_dict(openml_list, orient='index') # Transform to pandas
#datalist = datalist[datalist.status == 'active'] # Only use active (verified) datasets
datalist = datalist[datalist.NumberOfClasses>=2] # Only classification
print("{} {} classification datasets".format(len(datalist), status_type))

1038 active classification datasets


In [3]:
# Bookkeeping
data_names = {k: v for (k, v) in datalist[['did','name']].values} # dataset names
data_status = {k: 'OK' for k in datalist.index} # dataset status (OK or reason for removal)
datalist_full = datalist.copy()

## Step 1: Apply simple preconditions
- Number of observations larger than 500 (meaningful evaluations)
- Number of observations smaller than 100000 (keep runtime manageable)
- Number of features does not exceed 5000 (keep runtime manageable)
- The ratio of the minority class and the majority class is > 0.05 (severely imbalanced datasets complicate analysis)
- Number of values for categorical features must not exceed 100 (severely slows down some algorithms)
- Sparsely formatted data (requires special data readers)

In [4]:
# Apply preconditions
data_status.update({k: 'Too small' for k in datalist.index[datalist.NumberOfInstances<500]})
data_status.update({k: 'Too large' for k in datalist.index[datalist.NumberOfInstances>100000]})
data_status.update({k: 'High-dimensional' for k in datalist.index[datalist.NumberOfFeatures>2000]})
data_status.update({k: 'Extreme imbalance' for k in datalist.index[datalist.MinorityClassSize / datalist.MajorityClassSize < 0.05]})
#data_status.update({k: 'Too many categories' for k in datalist.index[datalist.MaxNominalAttDistinctValues > 100]})
data_status.update({k: 'Sparse format' for k in datalist.index[datalist.format == 'Sparse_ARFF']})

# Filter dataset list
datalist = datalist[pd.Series({k:(v=='OK') for k,v in data_status.items()})] 

# Status update
[[x,list(data_status.values()).count(x)] for x in set(data_status.values())]

[['Too small', 340],
 ['OK', 311],
 ['Sparse format', 32],
 ['High-dimensional', 65],
 ['Too large', 110],
 ['Extreme imbalance', 180]]

## Step 2: Filter out special datasets
- Artificial datasets (may bias the results)
- Time series dataset (cannot use random sampling for evaluation)
- Text data (contains string features which need additional preprocessing)
- Multilabel data (multiple targets need to be predicted)
- Derived versions of datasets (with additional preprocessing)
- Datasets where the intended classification target is unclear
- Binarized regression problems
- Unknown origin (no description how data was collected and what the problem is)
- Grouped data (instances form groups (blocks) and can't be randomly sampled)

In [5]:
# Get lists of special datasets
artificial_set = set(oml.datasets.list_datasets(tag="artificial").keys())
timeseries_set = set(oml.datasets.list_datasets(tag="time_series").keys())
text_set = set(oml.datasets.list_datasets(tag="text_data").keys())
multilabel_set = set(oml.datasets.list_datasets(tag="multi_label").keys())
derived_set = set(oml.datasets.list_datasets(tag="derived").keys())
unspecified_set = set(oml.datasets.list_datasets(tag="unspecified_target_feature").keys())
binarized_set = set(oml.datasets.list_datasets(tag="binarized_regression_problem").keys())
unknown_set = set(oml.datasets.list_datasets(tag="origin_unknown").keys())
grouped_set = set(oml.datasets.list_datasets(tag="grouped_data").keys())

data_status.update({k: 'Artificial data' for k in artificial_set})
data_status.update({k: 'Time series data' for k in timeseries_set})
data_status.update({k: 'Text data' for k in text_set})
data_status.update({k: 'Multi-label data' for k in multilabel_set})
data_status.update({k: 'Derived (non-original) data' for k in derived_set})
data_status.update({k: 'Unspecified target feature' for k in unspecified_set})
data_status.update({k: 'Binarized regression problem' for k in binarized_set})
data_status.update({k: 'Unknown origin' for k in unknown_set})
data_status.update({k: 'Grouped data' for k in grouped_set})
#data_status.update({k: 'OpenML100' for k in openml100_set})

# Filter dataset list
datalist = datalist[pd.Series({k:(v=='OK') for k,v in data_status.items()})] 

# Status update
[[x,list(data_status.values()).count(x)] for x in set(data_status.values())]

/home/vanrijn/projects/pythonvirtual/pimp/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


[['Binarized regression problem', 88],
 ['Unknown origin', 8],
 ['Time series data', 6],
 ['Too small', 339],
 ['Artificial data', 203],
 ['Text data', 2],
 ['Multi-label data', 7],
 ['OK', 129],
 ['Sparse format', 32],
 ['High-dimensional', 64],
 ['Unspecified target feature', 5],
 ['Too large', 23],
 ['Grouped data', 2],
 ['Derived (non-original) data', 40],
 ['Extreme imbalance', 113]]

## Step 3: Remove alternative versions of datasets
- Remove binarized versions of multi-class datasets
- Check other possible duplicates

In [6]:
# Sorting makes things easier
# We need the full list because there may be binarized versions of already removed datasets
datalist_full = datalist_full.sort_values(by=['name','NumberOfClasses'], ascending=[True, False])

checked_datasets = {
    40979: [1022, 20], # Correct version of mfeat-pixel
    40984: [958, 36], # Correct version of segment
    40994: [40990, 40989, 1467], # Correct version of climate-model-simulation-crashes
    1590: [179], # Correct version of adult
    40983: [1570], # Correct version of wilt
    40945: [40704], # Correct version of Titanic
    772: [948], # Correct version of classification version of the quake dataset
    40966: [40965, 40964], # Correct version of MiceProtein (needed when including in_preparation datasets)
    40982: [40973, 1504], # Correct version of steel-plates-fault (needed when including in_preparation datasets)
    23380: [1024, 473], # Correct version of cjs (needed when including in_preparation datasets)
    40597 : [40733], # Correct version of yeast (needed when including in_preparation datasets)
    1046 : [40829], # Correct version of mozilla4 (needed when including in_preparation datasets)
    # : [40958], # Correct version of Bankdata (needed when including in_preparation datasets)
    
}
duplicates_of = {}
# Mark the duplicates of datasets where we know which version is the correct one!
for cd in checked_datasets:
    for dup_id in checked_datasets[cd]:
        duplicates_of[dup_id] = cd

data_unique = {}
for index, row in datalist_full.iterrows():
    if row['did'] in duplicates_of:
        data_status[row['did']] = 'Duplicate of %d' % duplicates_of[row['did']]
    elif row['did'] in checked_datasets and data_status[row['did']] in ('OK', 'Possible duplicate'):
        data_status[row['did']] = 'OK'
    elif row['name'] not in data_unique:
        data_unique[row['name']] = row
    else:
        previous = data_unique[row['name']]
        if previous['NumberOfClasses'] > 2 and row['NumberOfClasses'] == 2:
            data_status[row['did']] = 'Binarized version of multiclass dataset'
        elif data_status[row['did']] in ('OK', 'Possible duplicate'):
            data_status[row['did']] = 'Possible duplicate'

# Filter dataset list
datalist = datalist[pd.Series({k:(v=='OK') for k,v in data_status.items()})] 
               
# Status update
[[x,list(data_status.values()).count(x)] for x in set(data_status.values())]

/home/vanrijn/projects/pythonvirtual/pimp/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


[['Text data', 2],
 ['Binarized version of multiclass dataset', 77],
 ['OK', 91],
 ['Grouped data', 2],
 ['Extreme imbalance', 112],
 ['Duplicate of 1590', 1],
 ['Multi-label data', 6],
 ['Sparse format', 31],
 ['Duplicate of 40983', 1],
 ['High-dimensional', 64],
 ['Unspecified target feature', 5],
 ['Duplicate of 40979', 2],
 ['Duplicate of 40984', 2],
 ['Unknown origin', 8],
 ['Duplicate of 772', 1],
 ['Too small', 298],
 ['Artificial data', 202],
 ['Binarized regression problem', 86],
 ['Time series data', 6],
 ['Duplicate of 40994', 1],
 ['Duplicate of 40982', 1],
 ['Too large', 23],
 ['Duplicate of 40945', 1],
 ['Derived (non-original) data', 38]]

In [7]:
# These need to be checked
[k for k,v in data_status.items() if v=='Possible duplicate']

[]

## Step 4: Remove trivial datasets
- See if a model (e.g. random forest) based on 1 feature can get perfect CV performance

In [ ]:
datasets = [k for k,v in data_status.items() if v=='OK']

max_score_per_dataset = {}
for idx, dataset_id in enumerate(datasets):
    try:
        
        dataset = oml.datasets.get_dataset(dataset_id)
        print('processing dataset', dataset_id, dataset.name, '(',idx+1, '/', len(datasets), ')')
        if dataset.default_target_attribute is None:
            data_status[dataset_id] = 'No target specified'
            print('No target')
            continue
        X, y = dataset.get_data(target=dataset.default_target_attribute)

        print('hotencoding dataset ... (for too much features)')
        categorical_indices = dataset.get_features_by_type('nominal', [dataset.default_target_attribute])
        clf = sklearn.pipeline.Pipeline(steps=[('imputer', ConditionalImputer(strategy='median', strategy_nominal='mean', categorical_features=categorical_indices, fill_empty=-1)), 
                                               ('encoder', sklearn.preprocessing.OneHotEncoder(categorical_features=categorical_indices))])
        hotencoded = clf.fit_transform(X)
        if hotencoded.shape[1] > 2000:
            data_status[dataset_id] = 'hotencoded-dimensionality'
            print(dataset.name, 'too high hot-encoded dimensionality', len(hotencoded))
            continue

        print('building decision stump')
        clf = sklearn.pipeline.Pipeline(steps=[('imputer', sklearn.preprocessing.Imputer(strategy='median')), 
                                               ('classifier', sklearn.tree.DecisionTreeClassifier(max_depth=1))])
        _ = clf.fit(X, y)
        score = clf.score(X, y)
        
        print('obtaining cv task .. ')
        # TODO

        max_score_per_dataset[dataset_id] = {
            'score': score,
            'name': dataset.name
        }
    except ValueError as e:
        data_status[dataset_id] = 'Python ValueError'
        print(dataset_id, e)
        continue
    except Exception as e:
        data_status[dataset_id] = 'Python Exception'
        print(dataset_id, e)
        continue
    
    
    if max_score_per_dataset[dataset_id]["score"] == 1.00:
        data_status[dataset_id] = 'Too easy'
        print("Dataset ", dataset.name, "is too easy.")
    
results = pd.DataFrame(max_score_per_dataset).transpose()

processing dataset 3 kr-vs-kp ( 1 / 91 )
hotencoding dataset ... (for too much features)
building decision stump
obtaining cv task .. 
processing dataset 6 letter ( 2 / 91 )
hotencoding dataset ... (for too much features)
building decision stump
obtaining cv task .. 
processing dataset 11 balance-scale ( 3 / 91 )
hotencoding dataset ... (for too much features)
building decision stump
obtaining cv task .. 
processing dataset 12 mfeat-factors ( 4 / 91 )
hotencoding dataset ... (for too much features)
building decision stump
obtaining cv task .. 
processing dataset 14 mfeat-fourier ( 5 / 91 )
hotencoding dataset ... (for too much features)
building decision stump
obtaining cv task .. 
processing dataset 15 breast-w ( 6 / 91 )
hotencoding dataset ... (for too much features)
building decision stump
obtaining cv task .. 
processing dataset 16 mfeat-karhunen ( 7 / 91 )
hotencoding dataset ... (for too much features)
building decision stump
obtaining cv task .. 
processing dataset 18 mfeat-mor

In [ ]:
results.sort_values(by='score')

### Results
Final list of selected datasets:

In [ ]:
final_datasets = [k for k,v in data_status.items() if v=='OK']
print('{} datasets selected'.format(len(final_datasets)))
{k:v for k,v in data_names.items() if k in final_datasets}

Passed all tests, but not in OpenML100:

In [ ]:
openml100_set = set(oml.datasets.list_datasets(tag="OpenML100").keys()) # OpenML100

new_datasets = [k for k,v in data_status.items() if v=='OK' and k not in openml100_set]
{k:v for k,v in data_names.items() if k in new_datasets}

Datasets tagged with OpenML100 that did not pass all tests:

In [ ]:
new_datasets = [k for k,v in data_status.items() if k in openml100_set and v!='OK']
{k:v for k,v in data_names.items() if k in new_datasets}

Reasons to exclude datasets:

In [ ]:
v = {}
for key, value in sorted(data_status.items()):
    v.setdefault(value, []).append(key)
{k:str(v) for k,v in v.items()}

## Difference to google doc

In [ ]:
names_selected = {v for k,v in data_names.items() if k in final_datasets}
google_doc = {
    "phoneme","breast-w","Australian","banknote-authentication","eeg-eye-state","electricity",
    "analcatdata_dmft","higgs","blood-transfusion-service-center","ilpd","steel-plates-fault",
    "satimage","mfeat-morphological","balance-scale","credit-a","cmc","diabetes","wilt","MagicTelescope",
    "vowel","pc3","adult","pc4","ada_agnostic","GesturePhaseSegmentationProcessed","PhishingWebsites",
    "bank-marketing","cardiotocography","climate-model-simulation-crashes","first-order-theorem-proving",
    "wall-robot-navigation","dresses-sales","sick","waveform-5000","wdbc","car","tic-tac-toe","mfeat-zernike",
    "segment","connect-4","kc2","jm1","pc1","kc1","qsar-biodeg","eucalyptus","credit-g","pendigits","vehicle",
    "letter","optdigits","mfeat-fourier","mfeat-karhunen","kr-vs-kp","ozone-level-8hr","sylva_agnostic",
    "nomao","spambase","splice","mushroom","cylinder-bands","SpeedDating","texture","mfeat-factors","collins",
    "mnist_784","gina_agnostic","Bioresponse","Internet-Advertisements","semeion","soybean","madelon","har",
    "isolet","micro-mass","cnae-9","MiceProtein","one-hundred-plants-margin","one-hundred-plants-shape",
    "one-hundred-plants-texture","mfeat-pixel",
}

In [ ]:
print("New ones!")
for d in sorted(names_selected - google_doc):
    print('  ', d)
print("Dropped ones!")
for d in sorted(google_doc - names_selected):
    print('  ', d)
print('Difference')
for d in sorted(google_doc ^ names_selected):
    print('  ', d)